In [ ]:
!git clone -b aurora https://github.com/tolios/query2vec.git

In [ ]:
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric --no-index --find-links=file:///kaggle/input/torch-geometric
!pip install mlflow==2.5

In [ ]:
!wget https://download.microsoft.com/download/8/7/0/8700516A-AB3D-4850-B4BB-805C515AECE1/FB15K-237.2.zip
!unzip /kaggle/working/FB15K-237.2.zip
!mv /kaggle/working/Release /kaggle/working/FB15k_237

In [ ]:
bash_script = '''
#!/usr/bin/env bash

dataset_path="/kaggle/working/FB15k_237/"
qa_folder="qa_2p_10k" #149689
train_path=$dataset_path"train.txt"
val_path=$dataset_path"valid.txt"
test_path=$dataset_path"test.txt"
train_query_orders="[[(1, -1), (2, 10000), (2, 149689), (3, 149689)]]"
val_query_orders="[[(1, -1), (2, 5000), (2, 5000), (3, 5000)]]"
test_query_orders="[[], [(2, 5000)], [(2, 5000)], [(3, 5000)], [(3, 5000)], [(3, 5000)]]"
include_train='[["1p", "2p", "2i", "3i"]]'
include_val='[["1p", "2p", "2i", "3i"]]'
include_test='[["1p"], ["2p"], ["2i"], ["3i"], ["pi"], ["ip"]]'

python ./query2vec/graph.py $train_path $val_path $test_path \
            --qa_folder=$qa_folder --train_query_orders="$train_query_orders" \
            --val_query_orders="$val_query_orders" --test_query_orders="$test_query_orders" \
            --include_train="$include_train" --include_val="$include_val" --include_test="$include_test" \
            --add_inverse=true

test_filter=$dataset_path$qa_folder"/filter.pkl"
val_filter=$dataset_path$qa_folder"/val_filter.pkl"
train_ds=$dataset_path$qa_folder"/train_qa_1.txt"
val_ds=$dataset_path$qa_folder"/val_qa_1.txt"

python ./query2vec/create_filter.py $test_filter $train_ds $val_ds $val_filter
'''


# Save the Bash script to a file
with open('script.sh', 'w') as f:
    f.write(bash_script)

!bash /kaggle/working/script.sh

In [ ]:
import json

model_json = '''{
    "model_type": "MDRTkernel",
    "model_params": {
        "kernel": "rgcn",
        "T": 0.1,
        "T_emb": 0.25,
        "aggregation": "sum",
        "dynamic": false,
        "conv_dims" : [300],
        "num_bases" : null,
        "num_blocks" : null,
        "linear_dims": [],
        "heads": 1,
        "emb_dim": 200,
        "p": 0.0,
        "margin": 1.0
    }
}
'''

config_json = '''{
    "experiment": "query2vec_init",
    "experiment_id": null,
    "run": "qa_MDRTkernel_full_1",
    "config": {
        "seed": 42,
        "train_data": "./FB15k_237/qa_2p_10k/train_qa_1.txt",
        "val_data": "./FB15k_237/qa_2p_10k/val_qa_1.txt",
        "epochs": 20,
        "train_batch_size": 1024,
        "val_batch_size": 1024,
        "num_negs": 50,
        "lr": 0.0001,
        "wd": 0.000001,
        "patience": 2,
        "val_every": 1,
        "scheduler_patience": 4,
        "scheduler_factor": 0.1,
        "scheduler_threshold": 0.1,
        "pretrained": null
    }
}
'''

# Parse JSON strings into Python dictionaries
model_json = json.loads(model_json)
config_json = json.loads(config_json)

# Write JSON data to files
with open('model.json', 'w') as f:
    json.dump(model_json, f)
    
with open('train_config.json', 'w') as f:
    json.dump(config_json, f) 

In [ ]:
!python /kaggle/working/query2vec/run.py /kaggle/working/ --all_scaled

In [ ]:
#download runs...

!zip -r /kaggle/working/mlruns.zip /kaggle/working/mlruns